**This file contains:**<br/>
The closed-form solutions for the pricing of European Options, American Options, Asian Options, Spread Options, Heat Rate Options, and Implied Volatility.


In [1]:
# import the necessary libaries
import unittest
import math
import numpy as np
from scipy.stats import norm, mvn